In [28]:
# Author: LEITE, G. B. S.
# Description: Applying the Simple Additive Weighting (SAW) method to a decision matrix

In [6]:
import pandas as pd
import numpy as np
from operator import itemgetter

In [32]:
# Decision matrix definition
m = [[1.2, 25, 8, 58000, 7, 5, 1126],
     [1.65, 40, 7, 31000, 7, 8, 334],
     [1.27, 20, 9, 45000, 9, 6, 227],
     [1.77, 25, 7, 50000, 8, 8, 550]]
print(f'Decision matrix: {m}')

# Criterion status definition, so that: 0 corresponds to minimization criteria and 1 to maximization criteria
st = [0, 0, 1, 1, 1, 1, 0]
print(f'Criteria status: {st}')

# Definition of weights for each criterion
pesos = [0.18, 0.2, 0.1, 0.2, 0.12, 0.05, 0.15]
print(f'Criteria weights: {pesos}')

Decision matrix: [[1.2, 25, 8, 58000, 7, 5, 1126], [1.65, 40, 7, 31000, 7, 8, 334], [1.27, 20, 9, 45000, 9, 6, 227], [1.77, 25, 7, 50000, 8, 8, 550]]
Criteria status: [0, 0, 1, 1, 1, 1, 0]
Criteria weights: [0.18, 0.2, 0.1, 0.2, 0.12, 0.05, 0.15]


In [33]:
# As the first step to apply the SAW in a decision matrix is to normalize it, the normalization function is defined

def normalizar_01(matriz, status):
  """
 -> Function to calculate normalization of type 0-1 of a decision matrix.

   :param matriz: decision matrix.
   :param status: list containing the status of each criterion - if it is a maxization criterion, the list receives 1 and, if it is a minization criterion, the list receives 0.
   :return: transposed normalized matrix.

  """

  l = []
  for j in range(len(matriz[0])):
    linha = []
    for i in range(len(matriz)):
      linha.append(matriz[i][j])
    l.append(linha)
  
  m_final = []
  maior = menor = 0
  for i in range(0, len(status)):
    if status[i] == 0:
      for j in range(0, len(l[i])):
        if j == 0:
            maior = l[i][j]
            menor = l[i][j]    
        else:
            if l[i][j] > maior:
              maior = l[i][j]
            if l[i][j] < menor:
              menor = l[i][j]

      for j in range(0, len(l[i])):
        if l[i][j] == maior:
          l[i][j] = 0
        elif l[i][j] == menor:
          l[i][j] = 1
        else:
          l[i][j] = (maior - l[i][j])/(maior - menor)
      m_final.append(l[i])

    else:
      maior = menor = 0
      for j in range(0, len(l[i])):
        if j == 0:
            maior = l[i][j]
            menor = l[i][j]
        else:
            if l[i][j] > maior:
              maior = l[i][j]
            if l[i][j] < menor:
              menor = l[i][j]

      for j in range(0, len(l[i])):
        if l[i][j] == maior:
          l[i][j] = 1
        elif l[i][j] == menor:
          l[i][j] = 0
        else:
          l[i][j] = (l[i][j] - menor)/(maior - menor)

      m_final.append(l[i])

  return m_final

In [31]:
# Normalization of the decision matrix from the 0-1 method
m_norm01 = normalizar_01(m, st)
print(f'Transposed normalized matrix:\n{np.array(m_norm01)}')

Transposed normalized matrix:
[[1.         0.21052632 0.87719298 0.        ]
 [0.75       0.         1.         0.75      ]
 [0.5        0.         1.         0.        ]
 [1.         0.         0.51851852 0.7037037 ]
 [0.         0.         1.         0.5       ]
 [0.         1.         0.33333333 1.        ]
 [0.         0.88097887 1.         0.6407119 ]]


#### **Simple Additive Weighting (SAW) Method**


Let the matrix be normalized: $X =
 \begin{pmatrix}
  x_{1,1} & x_{1,2} & \cdots & x_{1,j} \\
  x_{2,1} & x_{2,2} & \cdots & x_{2,j} \\
  \vdots  & \vdots  & \ddots & \vdots  \\
  x_{i,1} & x_{i,2} & \cdots & x_{i,j}
 \end{pmatrix}$;

 and the weight vector: $w =
 \begin{pmatrix}
  w_{1} & w_{2} & \cdots & w_{k} \\
 \end{pmatrix}$, so that $\sum_{j=i}^k w_{j} = 1$. Calculate the value $v_{i}$ of each alternative $A_{i}$, so that:
 
 $v_{i} = \sum_{j=i}^k x_{ij}w_{j}$. The alternatives are ordered according to the ascending order of the vector $v = \begin{pmatrix}
v_{1} & v_{2} & \cdots & v_{i} \\
\end{pmatrix}$.

In [26]:
def saw(matriz, pesos):
  """
-> Function to evaluate alternatives using the Simple Additive Weighting (SAW) method.

   :param matriz: transposed normalized decision matrix.
   :param pes: list containing the weights of each criterion analyzed.
  
   The function returns a print with the classification of the evaluated alternatives.

  """
  
  c = 0
  m_p = []
  while c < len(pesos):                 # The decision matrix is weighted by the weights of their respective criteria
    col = []
    for i in matriz[c]:
      col.append(i*pesos[c])
    m_p.append(col[:])
    col.clear()
    c += 1
  
  dic = {}                              # A dictionary with the weighted matrix is created
  for i in range(0, len(m_p)):
    dic[i] = m_p[i]
  
  df = pd.DataFrame(dic)                # A data frame is generated
  df['avaliaçao_saw'] = df.sum(axis=1)  # and the terms of each row (that is, of each evaluated alternative) are added together
  
  l = []                                # A list with the rating is created
  for i in df['avaliaçao_saw']:
    l.append(i)

  dics = {}                             # A dictionary with the rating is also created
  for i in range(0, len(l)):
    dics[i] = l[i]

  print('='*26, end=' ')
  print('\033[91m'+"Ranking SAW"+'\033[0m', end=' ')
  print('='*26)

  ranking = sorted(dics.items(), key=itemgetter(1), reverse=True)         # And, finally, the function returns the result with the classification of alternatives
  for i, v in enumerate(ranking):
    print(f'{i+1}st place: Alternative {v[0]+1} with a {v[1]} point rating')

In [27]:
# Application of the SAW method in the normalized matrix
saw(m_norm01, pesos)

========================== Ranking SAW ==========================
1st place: Alternative 3 with a 0.8482651072124757 point rating
2st place: Alternative 1 with a 0.5800000000000001 point rating
3st place: Alternative 4 with a 0.4968475260577596 point rating
4st place: Alternative 2 with a 0.22004156665300628 point rating
